## Model

### Model overview

#### Assumptions
* Neuron values are either 0 or 1 (binary). 
* Weights are real values, which are maybe bounded. 
* Fixed bias to fix ideas (subject to change).

#### Architecture
Let $H(x)$ be the binary step function 
$$
H(x):=\begin{cases}1 \quad x\geq 0\\0 \quad x< 0.\end{cases}
$$

The *forward pass* of one layer reads 
$$
L_i(x,w) := H\left( \sum_{j}  w[i,j]\cdot x[j] - B \right),
$$
where $x$ are the inputs, or neuron values, and $w$ are the weights, that is the synaptic values. We denote by $B$ the bias, it is a positive constant.

From inputs to final outputs with $n$ layers, the forward pass looks like 
$$
x_0 \xrightarrow{w_0} x_1 \xrightarrow{w_1} x_2 \rightarrow \dots \rightarrow x_{n-1} \xrightarrow{w_{n-1}} x_n,
$$
with 
$$
x_i = L(x_{i-1}, w_{i-1}), \quad x_0 = \text{given inputs}.
$$
Again, $x$ has only entries in $\{0, 1\}$, and $w$ has floats as entries (bounded?).  

The *backward pass* is given by
$$
gx_n \xrightarrow{gw_{n-1}} gx_{n-1} \xrightarrow{gw_{n-2}} gx_{n-2} \rightarrow \dots \rightarrow gx_{1} \xrightarrow{gw_{0}} gx_0,
$$
where 
\begin{split}
gx_i &= G(gx_{i+1}, x_{i+1}, x_{i}, w_{i})\\
gw_i &= W(gx_{i+1}, x_{i+1}, x_i) \\
gx_n &= Neq(\text{desired outputs}, x_n).
\end{split}
We will define $G$, $W$, and $Neq$ in the following. 

As for the forward pass, $gx$ has entries in $\{0, 1\}$, and $gw$ has floats as entries. Moreover, the shape of $x_i$ equals the shape of $gx_i$, and the shape of $w_i$ equals the shape of $gw_i$.

### Backward pass formulas

#### Neq
For $x, y \in \{0, 1\}$, we set
$$
Neq(x, y) = \begin{cases} 0 \quad  &x=y, \\ 1 \quad &x\neq y. \end{cases} 
$$
This is generalized to $x$ and $y$ of the same shape by applying entry by entry. The signature is
```
Neq: (bool, shape) x (bool, shape) -> (bool, shape)  
```

#### W
Let $n$ be the shape of $x$ and $gx$, and let $m$ be the shape of $x'$. We define $W(gx, x, x')$, which is of shape $n\times m$, by 
$$
W(gx, x, x') := \left( gx[i] \cdot P(x[i], x'[j]) \right)_{i,j},
$$
where 
$$
P(0, 1) = 1, \quad P(1, 1) = -1, \quad P(0, 0) = Q, \quad P(1, 0) = -Q,  
$$
with a non-negative $Q<1$. The value of $Q$ is connected to the learning rate (reference???). In order to compute $P$, we can use 
$$
P(x,y) =  (1 - 2 \cdot x) \cdot (Q  + (1 - Q) \cdot y) 
$$ 

The signature is
```
W: (bool, n) x (bool, n) x (bool, m) -> ({-1, -Q, 0, Q, 1}, n x m).  
```

#### G
Let $n$ be the shape of $x$ and $gx$, and $m$ the shape of $x'$, then $w'$ must have shape $n\times m$. We define 
$$
G(gx, x, x', w') = \left(H\left( \sum_{i} gx[i]\cdot Seq(x[i], x'[j])  \cdot w[i,j] - B\right) \right)_{j},
$$
where 
$$
Seq(x,y) = (2x-1)\cdot (2y-1)  = \begin{cases} 1 \quad  &x=y, \\ -1 \quad &x\neq y. \end{cases}
$$
The signature is 
```
G: (bool, n) x (bool, n) x (bool, m) x (float, n x m) -> (bool, m).
```

### Pytorch implementation

In [1]:
import torch

#### Heaviside 

In [2]:
input = torch.tensor([-1, 0, 1], dtype=torch.float32)
value = torch.tensor([1], dtype=torch.float32)

def H(input):
    return torch.heaviside(input, value)

#### Forward pass

In [3]:
weights = torch.tensor([[0.2, 0.1, 1.1], [0, -0.4, 0.3]], dtype=torch.float32, requires_grad=True)
x = torch.tensor([0, 1, 1], dtype=torch.float32)
weights, x

(tensor([[ 0.2000,  0.1000,  1.1000],
         [ 0.0000, -0.4000,  0.3000]], requires_grad=True),
 tensor([0., 1., 1.]))

In [4]:
torch.matmul(weights, x)

tensor([ 1.2000, -0.1000], grad_fn=<MvBackward0>)

In [5]:
def forward_bool(weights, input, B):
    return H(torch.matmul(weights, input) - B)

#### Neq

In [6]:
t = torch.tensor([[0, 1, 1, 1]], dtype=torch.float32)
s = torch.tensor([[0, 0, 0, 1]], dtype=torch.float32)
s, t 

(tensor([[0., 0., 0., 1.]]), tensor([[0., 1., 1., 1.]]))

In [7]:
def neq(s,t):
    return 1-(s==t).to(torch.float32)

In [8]:
neq(s,t)

tensor([[0., 1., 1., 0.]])

#### W

In [9]:
x = torch.tensor([0,1], dtype=torch.float32)
y = torch.tensor([1,1,0], dtype=torch.float32, requires_grad=True)
x, y

(tensor([0., 1.]), tensor([1., 1., 0.], requires_grad=True))

In [10]:
Q = torch.tensor([0.8], dtype=torch.float32)

first_factor = 1 - 2*x
second_factor = Q + (1-Q) * y
first_factor, second_factor

(tensor([ 1., -1.]), tensor([1.0000, 1.0000, 0.8000], grad_fn=<AddBackward0>))

In [11]:
torch.mm(torch.reshape(first_factor, first_factor.shape+(1,)),  
         torch.reshape(second_factor, (1,)+second_factor.shape))

tensor([[ 1.0000,  1.0000,  0.8000],
        [-1.0000, -1.0000, -0.8000]], grad_fn=<MmBackward0>)

In [12]:
# bring in gx
gx = torch.tensor([1,0], dtype=torch.float32)
first_factor = gx * (1 - 2*x)

In [13]:
torch.mm(torch.reshape(first_factor, first_factor.shape+(1,)),  
         torch.reshape(second_factor, (1,)+second_factor.shape))

tensor([[1.0000, 1.0000, 0.8000],
        [-0.0000, -0.0000, -0.0000]], grad_fn=<MmBackward0>)

In [14]:
def W(gx, x, y, q):
    first_factor = gx * (1 - 2*x)
    second_factor = q + (1-q) * y
    return torch.mm(torch.reshape(first_factor, first_factor.shape+(1,)),  
             torch.reshape(second_factor, (1,)+second_factor.shape))

In [15]:
W(gx,x,y,Q)

tensor([[1.0000, 1.0000, 0.8000],
        [-0.0000, -0.0000, -0.0000]], grad_fn=<MmBackward0>)

#### G

In [16]:
x = torch.tensor([0,1], dtype=torch.float32)
y = torch.tensor([1,1,0], dtype=torch.float32, requires_grad=True)

seq_first_factor = 2*x - 1
seq_second_factor = 2*y - 1
seq_first_factor, seq_second_factor

(tensor([-1.,  1.]), tensor([ 1.,  1., -1.], grad_fn=<SubBackward0>))

In [17]:
def seq(x, y):
    seq_first_factor = 2*x - 1
    seq_second_factor = 2*y - 1
    return torch.mm(torch.reshape(seq_first_factor, seq_first_factor.shape+(1,)),  
             torch.reshape(seq_second_factor, (1,)+seq_second_factor.shape))

In [18]:
seq(x, y)

tensor([[-1., -1.,  1.],
        [ 1.,  1., -1.]], grad_fn=<MmBackward0>)

In [19]:
def G(gx, x, y, w, B):
    new_w = seq(x, y) * w 
    return H(torch.matmul(gx, new_w) - B)

In [20]:
B = torch.tensor([0.5])
print("gx : {} \n x: {} \n y: {}\n w: {} \n B: {}".format(gx, x, y, weights, B))
G(gx, x, y, weights, B)

gx : tensor([1., 0.]) 
 x: tensor([0., 1.]) 
 y: tensor([1., 1., 0.], requires_grad=True)
 w: tensor([[ 0.2000,  0.1000,  1.1000],
        [ 0.0000, -0.4000,  0.3000]], requires_grad=True) 
 B: tensor([0.5000])


tensor([0., 0., 1.], grad_fn=<NotImplemented>)

#### Autograd function

In [21]:
class LinearBool(torch.autograd.Function):
    @staticmethod
    # B and Q are added as tensors
    def forward(ctx, input, weight, b, q):
        output = forward_bool(weight, input, b)
        ctx.save_for_backward(input, weight, b, q, output)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, weight, b, q, output = ctx.saved_tensors

        grad_input = grad_weight = grad_b = grad_q = None

        grad_input = G(grad_output, output, input, weight, b)
        grad_weight = W(grad_output, output, input, q)

        return grad_input, grad_weight, grad_b, grad_q
        

##### Tests

In [22]:
# Forward

linear = LinearBool.apply
#print(y, weights, B, Q)
output = linear(y, weights, B, Q)
output

tensor([0., 0.], grad_fn=<LinearBoolBackward>)

In [23]:
# Backward
desired_output = torch.tensor([1, 1], dtype=torch.float32)
gradient = neq(output, desired_output)

output.backward(gradient)

In [24]:
weights.grad, y.grad

(tensor([[1.0000, 1.0000, 0.8000],
         [1.0000, 1.0000, 0.8000]]),
 tensor([0., 0., 1.]))

#### Learning tools

In [25]:
import numpy as np

def learn(inputs, desired_outputs, model, weights, bias, learning_rate, 
            binarizatio_fn=lambda input: torch.heaviside(input, torch.tensor([0.])),
            loss_gradient = neq):

    for w in weights:
        w.grad = None

    for b in bias:
        b.grad = None

    eq_counter = 0
    for i, o in zip(inputs, desired_outputs):
        output = model(i)
        output_bn = binarizatio_fn(output)

        if torch.all(o==output_bn):
            eq_counter += 1
        else:
            gradient = loss_gradient(o, output_bn)
            
            output.backward(gradient)
            with torch.no_grad():
                for w in weights:
                    w += learning_rate * w.grad
                    w.grad = None
                for b in bias:
                    b += learning_rate * b.grad
                    b.grad = None
                    
    if eq_counter == len(inputs):
        return "Done"

#### Comparison

In order to compare performance with a non binary approach, we introduce a simple linear layer with constant bias and ReLU.  

In [26]:
def linear_compare(y, weights, B):
    l = torch.matmul(weights, y) - B
    return torch.nn.ReLU()(l)

def loss_gradient(o, output_bn):
    return o-output_bn

### Examples

All examples start with random initialisation of weights in the range $(0, 1)$.

#### Copy one neuron

Architecture: `input_neuron -> output_neuron` 

Desired behavior: `0->0, 1->1`

Explicitly, for forward $y=H(x\cdot w-B)$. If $x=0$ then $y=0$, which is desired behavior. Suppose $x=1$ and $y=0$. Desired value is $y'=1$. For backward: the first gradient is $gy= Eq(y', y)=1$. The weight gradient reads $gw = W(gy, y, x)= 1$, pointing to increasing $w$, which is correct behavior. 

Once correct behavior is established, the weight gradients are zero. 

In [27]:
y = torch.tensor([1.], requires_grad=True)
desired_output = torch.tensor([1.], requires_grad=True)
# start with negative weight, it will become positive after learning 
w = torch.tensor([-0.6], requires_grad=True)
print("Starting with weight {}".format(w))
gradient = None
limit = 10
counter = 0

while gradient != torch.tensor([0.]) and counter<limit:
    output = linear(y, w, B, Q)
    print("Output: {}".format(output))

    gradient = neq(output, desired_output)
    output.backward(gradient)
    print("Weight gradient: {}".format(w.grad))

    with torch.no_grad():
        w += w.grad
    print("Changing weight to {}".format(w))
    w.grad = None

    counter += 1

Starting with weight tensor([-0.6000], requires_grad=True)
Output: tensor([0.], grad_fn=<LinearBoolBackward>)
Weight gradient: tensor([1.])
Changing weight to tensor([0.4000], requires_grad=True)
Output: tensor([0.], grad_fn=<LinearBoolBackward>)
Weight gradient: tensor([1.])
Changing weight to tensor([1.4000], requires_grad=True)
Output: tensor([1.], grad_fn=<LinearBoolBackward>)
Weight gradient: tensor([0.])
Changing weight to tensor([1.4000], requires_grad=True)


#### Copy one neuron with a neuron in between

Architecture: `input_neuron -> other_neuron -> output_neuron` 

Desired behavior: `0->0, 1->1`

The backpropagation of 
$$
x_\text{in} \xrightarrow{w_\text{in}} x_\text{other} \xrightarrow{w_\text{other}} x_{\text{out}}
$$
is computed as follows
$$
gx_\text{out} \xrightarrow{gw_\text{other}} gx_\text{other} \xrightarrow{gw_\text{in}} gx_\text{in}$$
Independent of the $x_{\text{other}}$ values, $gw_\text{other}$ will be positive if the desired value for $x_\text{out}$ is not reached. Once $w_\text{other}$ is large enough, $gx_\text{other}$ will become positive if $x_\text{out}=0$ is not desired and $x_{\text{other}}=0$. This will push $gw_\text{in}$ to positive, fixing $w_\text{in}$. The model will solve the task.  



In [28]:
y = torch.tensor([1.], requires_grad=True)
desired_output = torch.tensor([1.], requires_grad=True)

# start with negative weight, it will become positive after learning 
w_in = torch.tensor([-0.6], requires_grad=True)
w_other = torch.tensor([-0.9], requires_grad=True)
weights = [w_in, w_other]
print("Starting with weights {}".format(weights))

def model(input): 
    return linear(linear(input, w_in, B, Q), w_other, B, Q)

gradient = None
limit = 10
counter = 0

while gradient != torch.tensor([0.]) and counter < limit:
    output = model(y)
    print("Output: {}".format(output))

    gradient = neq(output, desired_output)
    output.backward(gradient)
    print("Weight gradients: {}".format([w.grad for w in weights]))

    with torch.no_grad():
        for w in weights:
            w += w.grad
            w.grad = None            
    print("Changing weight to {}".format(weights))
 
    counter += 1

Starting with weights [tensor([-0.6000], requires_grad=True), tensor([-0.9000], requires_grad=True)]
Output: tensor([0.], grad_fn=<LinearBoolBackward>)
Weight gradients: [tensor([0.]), tensor([0.8000])]
Changing weight to [tensor([-0.6000], requires_grad=True), tensor([-0.1000], requires_grad=True)]
Output: tensor([0.], grad_fn=<LinearBoolBackward>)
Weight gradients: [tensor([0.]), tensor([0.8000])]
Changing weight to [tensor([-0.6000], requires_grad=True), tensor([0.7000], requires_grad=True)]
Output: tensor([0.], grad_fn=<LinearBoolBackward>)
Weight gradients: [tensor([1.]), tensor([0.8000])]
Changing weight to [tensor([0.4000], requires_grad=True), tensor([1.5000], requires_grad=True)]
Output: tensor([0.], grad_fn=<LinearBoolBackward>)
Weight gradients: [tensor([1.]), tensor([0.8000])]
Changing weight to [tensor([1.4000], requires_grad=True), tensor([2.3000], requires_grad=True)]
Output: tensor([1.], grad_fn=<LinearBoolBackward>)
Weight gradients: [tensor([0.]), tensor([0.])]
Changi

#### Always disagree with one neuron

Architecture: `input_neuron -> output_neuron` 

Desired behavior: `1->0`. Note that `0->0` is automatic. 

Explicitly, for forward $y=H(x\cdot w-B)$. If $x=1$ and $y=1$, then the weight gradient is
$
gw = -1
$
pushing the weights in the negative direction, which is correct behavior. 

In [29]:
inputs = [torch.tensor([1.], requires_grad=True)]
desired_outputs = [torch.tensor([0.])]
weights = [torch.tensor([1.], requires_grad=True)]

def model(input):
    return linear(input, weights[0], B, Q)

for i in range(10):
    learn(inputs, desired_outputs, model, weights, [], learning_rate=0.1)

print("New weights: {}".format(weights))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))

New weights: [tensor([0.5000], requires_grad=True)]
[1.] -> [0.]


#### Switch positions

Architecture: `input_neuron_0, input_neuron_1  -> output_neuron_0, output_neuron_1` 

Desired behavior: `I:(1,0)->(0,1), II:(0,1)->(1,0), III:(1,1)->(1,1)`

Explicitly, for forward $y_i=H(x_0\cdot w_{i,0} + x_1\cdot w_{i,1} - B)$. Let's consider the weights one by one:
* $w_{0,0}$: since `I` and `III` send contradictory impulses at first, no changes.
* $w_{0,1}$: increasing, so it will fix `II` and have a strong angle on `III`. This will allow $w_{0,0}$ to focus on `I` and fix it. 
* $w_{1,0}$: Same as $w_{0,1}$.
* $w_{1,1}$: Same as $w_{0,0}$.

In [30]:
inputs = [torch.tensor([1., 0.], requires_grad=True), 
          torch.tensor([0., 1.], requires_grad=True),
          torch.tensor([1., 1.], requires_grad=True)]
desired_outputs = [torch.tensor([0., 1.]), torch.tensor([1., 0.]), torch.tensor([1., 1.])]

# initialization must be random
weights = [torch.tensor([[0.1, -0.2], 
                         [-0.22, -0.01]], requires_grad=True)]

def model(input):
    return linear(input, weights[0], B, Q)

for i in range(12):
    done = learn(inputs, desired_outputs, model, weights, [],  learning_rate=0.1)
    if done=="Done":
        print("Done after epoch {}".format(i))
        break

print("New weights: {}".format(weights))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))

Done after epoch 11
New weights: [tensor([[0.4800, 0.5400],
        [0.5800, 0.4900]], requires_grad=True)]
[1. 0.] -> [0. 1.]
[0. 1.] -> [1. 0.]
[1. 1.] -> [1. 1.]


In [31]:
# Comparison with ReLUs 

# initialization must be random
# Following weights won't work, because ReLU does not propagate grads if value = 0
# weights = [torch.tensor([[0.1, -0.2], 
#                         [-0.22, -0.01]], requires_grad=True)]
weights = [torch.tensor([[0.1, 0.13], 
                            [0.05, 0.07]], requires_grad=True)]

#bias = [torch.tensor([0.5], requires_grad=True)]
bias = []
binarization_fn = lambda input: torch.heaviside(input-0.2, torch.tensor([0.]))

def model(input):
    return linear_compare(input, weights[0], torch.tensor([0.])) 

for i in range(20):
    done = learn(inputs, desired_outputs, model, weights, bias, learning_rate=0.01, 
                    binarizatio_fn=binarization_fn, loss_gradient=loss_gradient)
    #done = learn(inputs, desired_outputs, model, weights, bias, learning_rate=0.1)
    if done=="Done":
        print("Done after epoch {}".format(i))
        break

print("New weights: {}".format(weights))
print("New bias: {}".format(bias))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), binarization_fn(model(input)).numpy()))

Done after epoch 12
New weights: [tensor([[0.1000, 0.2000],
        [0.2000, 0.1000]], requires_grad=True)]
New bias: []
[1. 0.] -> [0. 1.]
[0. 1.] -> [1. 0.]
[1. 1.] -> [1. 1.]


#### Hidden representation

Architecture: `input_0, input_1  -> hidden_0, hidden_1, hidden_2 -> output` 

Desired behavior: `I:(1,0)->(1), II:(0,1)->(1), III:(1,1)->(0)`

This cannot be learned with one layer only, because `I` and `II` imply two weights that are larger than the bias. This contradicts `III`.

TODO

* If `learnig rate` is a multiple of `Q` or vice versa, then desired behaviour is not learned. Explanation?
* If `Q = 0` then desired behaviour is not learned. Explanation?

In [32]:
inputs = [torch.tensor([1., 0.], requires_grad=True), 
          torch.tensor([0., 1.], requires_grad=True),
          torch.tensor([1., 1.], requires_grad=True)]
desired_outputs = [torch.tensor([1.]), torch.tensor([1.]), torch.tensor([0.])]

use_Q = torch.tensor([0.1])

# initialization must be random
weights = [torch.tensor([[0.1, -0.2], 
                         [-0.22, -0.01],
                         [0.22, 0.01]], requires_grad=True), 
           torch.tensor([0.1, 0.16, -0.2], requires_grad=True)]

def model(input):
    hidden = linear(input, weights[0], B, use_Q)
    output = linear(hidden, weights[1], B, use_Q)
    return output

epochs = 10000

for i in range(epochs):
    done = learn(inputs, desired_outputs, model, weights, [], learning_rate=0.11)
    if done == "Done":
        print("Finished in epoch {}".format(i))
        break

print("New weights: {}".format(weights))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))

TypeError: learn() missing 1 required positional argument: 'bias'

#### Hidden representation 2

Architecture: `input_0, input_1  -> hidden_0, hidden_1 -> output` 

Desired behavior: `I:(1,0)->(1), II:(0,1)->(1), III:(1,1)->(0)`

This cannot be learned with one layer only, because `I` and `II` imply two weights that are larger than the bias. This contradicts `III`.

In [ ]:
inputs = [torch.tensor([1., 0.], requires_grad=True), 
          torch.tensor([0., 1.], requires_grad=True),
          torch.tensor([1., 1.], requires_grad=True)]
desired_outputs = [torch.tensor([1.]), torch.tensor([1.]), torch.tensor([0.])]

use_Q = torch.tensor([0.115])

# initialization must be random
weights = [torch.tensor([[0.1, 0.2], 
                         [0.22, 0.1]], requires_grad=True), 
           torch.tensor([0.1, 0.16], requires_grad=True)]

def model(input, B=torch.tensor([0.2])):
    hidden = linear(input, weights[0], B, use_Q)
    output = linear(hidden, weights[1], B, use_Q)
    return output

epochs = 10000

for i in range(epochs):
    done = learn(inputs, desired_outputs, model, weights, [], learning_rate=0.01)
    if done == "Done":
        print("Finished in epoch {}".format(i))
        break

print("New weights: {}".format(weights))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), model(input).numpy()))

Finished in epoch 143
New weights: [tensor([[0.2072, 0.2010],
        [0.1923, 0.0900]], requires_grad=True), tensor([ 0.2015, -0.0040], requires_grad=True)]
[1. 0.] -> [1.]
[0. 1.] -> [1.]
[1. 1.] -> [0.]


In [ ]:
### Comparison with ReLUs

inputs = [torch.tensor([1., 0.], requires_grad=True), 
          torch.tensor([0., 1.], requires_grad=True),
          torch.tensor([1., 1.], requires_grad=True)]
desired_outputs = [torch.tensor([1.]), torch.tensor([1.]), torch.tensor([0.])]

bias = []
binarization_fn = lambda input: torch.heaviside(input-0.2, torch.tensor([0.]))

# initialization must be random
weights = [torch.tensor([[0.1, 0.2], 
                         [0.22, 0.1]], requires_grad=True), 
           torch.tensor([0.1, 0.16], requires_grad=True)]

zero = torch.tensor([0.])

def model(input):
    hidden = linear_compare(input, weights[0], zero)
    output = linear_compare(hidden, weights[1], zero)
    return output

epochs = 10000

for i in range(epochs):
    done = learn(inputs, desired_outputs, model, weights, bias, learning_rate=0.05,
                    binarizatio_fn=binarization_fn, loss_gradient=loss_gradient)
    if done == "Done":
        print("Finished in epoch {}".format(i))
        break

print("New weights: {}".format(weights))

with torch.no_grad():
    for input in inputs:
        print("{} -> {}".format(input.numpy(), binarization_fn(model(input)).numpy()))

Finished in epoch 1572
New weights: [tensor([[-0.4186,  0.4077],
        [ 0.4946, -0.0263]], requires_grad=True), tensor([0.5427, 0.4234], requires_grad=True)]
[1. 0.] -> [1.]
[0. 1.] -> [1.]
[1. 1.] -> [0.]
